**Connecting to Google Colab Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


**Unzipping Dataset folder content**

In [ ]:
!unzip "/content/drive/MyDrive/CKplus.zip" -d "/content/dataset"

Installing DeepFace Dependencies

In [ ]:
# uninstalling TF packages for new version
!pip uninstall -y deepface tensorflow keras tf-keras retinaface mtcnn keras-nightly keras-preprocessing

#  Compatible stack for Colab (Py 3.12)
!pip install -q "tensorflow==2.19.0" "tf-keras==2.19.0"
# Keras 3 ships with TF 2.19 — no need to pin keras separately; let TF manage it.

# DeepFace version that exists on PyPI and works with TF 2.19 + tf-keras
!pip install -q "deepface==0.0.95" "retina-face==0.0.17" "mtcnn==0.1.1"

**Checking how many images would be needed for bias mitigation**

In [ ]:
import math
import numpy as np
import pandas as pd

#  INPUTS (edit paths)
FAIRNESS_CSV   = "/content/drive/MyDrive/All_Augmented_gender_fairness_multimodel_results.csv"
TRAIN_CSV      = "/content/dataset/processed/CKPLUS_Features_20250805_111308_faceprefixed_CLEAN.csv"
SELECT_DATASET = "CKplus"
#SELECT_MODEL   = "baseline"
#SELECT_MODEL   = "MobileNet_V3"
SELECT_MODEL   = "EfficientNet_B2"

#SELECT_MODEL   = "EfficientNet_B2"
EMOTIONS       = ["angry","disgust","fear","happy","sad","surprise","neutral"]

# Fairness target
TARGET_GAP_PPTS = 2.0   # acceptable gap (percentage points)

# Measured slopes (optional): columns = Dataset,Model,Emotion,Gender,images_added,gap_before,gap_after
MEASURED_SLOPE_CSV = None  # e.g. "/content/augment_round_effects.csv"

# Batch size you typically add per round
BATCH_ADD = 50

# Heuristic knobs
MIN_BOOTSTRAP_N        = 120    # if underrepresented bucket has < this, bring it up to this
BALANCE_RATIO_TARGET   = 0.9    # aim for N_under >= 0.9 * N_over
GAP_COEF               = 35.0   # images per ppt of excess gap (scaled by 1/sqrt(N_under))
ROUND_TO               = BATCH_ADD  # round recommendations to your batch size
MAX_ADD_PER_BUCKET     = 2000   # soft safety cap per bucket (tune to your budget)

#  LOAD DATA
fair = pd.read_csv(FAIRNESS_CSV)
fair = fair[(fair["Dataset"] == SELECT_DATASET) & (fair["Model"] == SELECT_MODEL)].copy()

# If accuracies are in %, convert to proportions
if fair["Man_Acc"].max() > 1.5 or fair["Woman_Acc"].max() > 1.5:
    fair["Man_Acc"]   = fair["Man_Acc"]   / 100.0
    fair["Woman_Acc"] = fair["Woman_Acc"] / 100.0

train = pd.read_csv(TRAIN_CSV)

# Normalize gender strings to Man/Woman
def canon_gender(x):
    s = str(x).strip().lower()
    if s in {"man","male","m"}: return "Man"
    if s in {"woman","female","f"}: return "Woman"
    if "man" in s: return "Man"
    if "woman" in s: return "Woman"
    return "Man"

train["gender"] = train["gender"].apply(canon_gender)
train["dominant_emotion"] = train["dominant_emotion"].str.lower()

# Count current training images per bucket (gender×emotion) — use 'split' to select training rows
counts = (train[train["dataset"].astype(str).str.lower() == "train"]
          .groupby(["gender","dominant_emotion"])["filename"]
          .count()
          .rename("train_count")
          .reset_index())

def current_N(gender, emotion):
    row = counts[(counts["gender"]==gender) & (counts["dominant_emotion"]==emotion)]
    return int(row["train_count"].iloc[0]) if not row.empty else 0

#  USE MEASURED SLOPE IF AVAILABLE, ELSE HEURISTIC
if MEASURED_SLOPE_CSV:
    slopes = pd.read_csv(MEASURED_SLOPE_CSV)
    slopes = slopes[(slopes["Dataset"]==SELECT_DATASET) & (slopes["Model"]==SELECT_MODEL)].copy()
    slopes["gap_before"] = slopes["gap_before"].astype(float)
    slopes["gap_after"]  = slopes["gap_after"].astype(float)
    # normalize to 'per 50 images' ppts gain
    slopes["gain_per_50_ppts"] = (slopes["gap_before"] - slopes["gap_after"]) * 100.0 \
                                 * (BATCH_ADD / slopes["images_added"].replace(0, np.nan))
else:
    slopes = None

def round_up_to(x, base):
    return int(math.ceil(float(x) / base) * base)

def estimate_needed_images(gap_ppts, N_under, N_over, gain_per_50_ppts=None, target_gap_ppts=TARGET_GAP_PPTS):
    # If we have a measured slope, use it (bounded by cap)
    if gain_per_50_ppts is not None and gain_per_50_ppts > 1e-6:
        remaining = max(0.0, gap_ppts - target_gap_ppts)
        batches = math.ceil(remaining / gain_per_50_ppts)
        return min(batches * BATCH_ADD, MAX_ADD_PER_BUCKET)

    # 1) Bootstrap floor
    if N_under < MIN_BOOTSTRAP_N:
        need = MIN_BOOTSTRAP_N - N_under
        return min(round_up_to(need, ROUND_TO), MAX_ADD_PER_BUCKET)

    # 2) Balance need (toward ratio target)
    balance_target = BALANCE_RATIO_TARGET * max(N_over, 1)
    balance_need = max(0, int(math.ceil(balance_target - N_under)))

    # 3) Gap-driven need (diminishing with sqrt(N_under))
    gap_excess = max(0.0, gap_ppts - target_gap_ppts)
    if gap_excess <= 0:
        gap_need = 0
    else:
        scale = 1.0 / max(1.0, math.sqrt(N_under))  # more when bucket is small
        gap_need = int(math.ceil(GAP_COEF * gap_excess * scale))

    # Combine: take the larger of balance or gap need
    need = max(balance_need, gap_need)
    if need <= 0:
        return 0

    return min(round_up_to(need, ROUND_TO), MAX_ADD_PER_BUCKET)

#  BUILD RECOMMENDATION TABLE
rows = []
for emo in EMOTIONS:
    r = fair[fair["Emotion"] == emo]
    if r.empty:
        continue
    a_m = float(r["Man_Acc"].values[0])
    a_w = float(r["Woman_Acc"].values[0])

    # handle NaNs safely
    if np.isnan(a_m) or np.isnan(a_w):
        gap = float("nan")
        under_gender = "Woman"  # arbitrary; we’ll recommend 0 in NaN case below
    else:
        gap = abs(a_m - a_w) * 100.0  # ppts
        under_gender = "Man" if a_m < a_w else "Woman"

    over_gender  = "Woman" if under_gender == "Man" else "Man"
    n_under = current_N(under_gender, emo)
    n_over  = current_N(over_gender,  emo)

    gain_p50 = None
    if slopes is not None:
        r2 = slopes[(slopes["Emotion"].str.lower()==emo) & (slopes["Gender"]==under_gender)]
        if not r2.empty and np.isfinite(r2["gain_per_50_ppts"].mean()):
            gain_p50 = float(r2["gain_per_50_ppts"].mean())

    if np.isnan(gap):
        rec = 0  # not enough eval data to compute gap; keep 0 or choose a small bootstrap if you prefer
        gap_out = float("nan")
    else:
        rec = estimate_needed_images(
            gap_ppts=gap, N_under=n_under, N_over=n_over,
            gain_per_50_ppts=gain_p50, target_gap_ppts=TARGET_GAP_PPTS
        )
        gap_out = round(gap, 2)

    rows.append({
        "Emotion": emo,
        "Gap_ppts": gap_out,
        "Under_gender": under_gender,
        "Train_N_under": n_under,
        "Train_N_over":  n_over,
        "Gain_per_50_ppts(measured)": (None if gain_p50 is None else round(gain_p50, 3)),
        "Recommended_add_images": int(rec)
    })

recommend = pd.DataFrame(rows).sort_values(["Gap_ppts"], ascending=False, na_position="last").reset_index(drop=True)
print(recommend.to_string(index=False))

recommend.to_csv("bias_reduction_recommendations.csv", index=False)
print("\nSaved → bias_reduction_recommendations.csv")

**Data_Augumentation**

In [ ]:
#
# Conditional Augmentation (One-Shot Plan, Resume-Safe, Fast)
# - Uses a fixed per-model augmentation plan you provided
# - Generates images with SD v1.5 (fast) or SDXL (toggleable)
# - Verifies + crops with DeepFace (MTCNN)
# - RESIZES CROPPED FACES TO 224x224 (CKplus requirement)
# - Saves resume state to avoid losing progress on disconnect
# - Merges into flat train folder + appends to train CSV (fixed schema)
#

# !pip install -q diffusers==0.27.2 transformers accelerate safetensors tqdm pandas opencv-python deepface==0.0.93 mtcnn

import os, shutil, cv2, numpy as np, pandas as pd, torch
from tqdm import tqdm
from typing import Dict, Tuple, List
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from deepface import DeepFace

#
# CONFIG DATASET PATHS(CKplus, AffectNet, CelebA, FER2013+)
# ----------------
TRAIN_IMG_DIR  = "/content/dataset/processed/train"   # flat folder of training images
TRAIN_CSV      = "/content/dataset/processed/CKPLUS_Features_20250805_111308_faceprefixed_CLEAN.csv"

SELECT_DATASET = "CKplus"            # Dataset Selection : CKplus
SELECT_MODEL   = "baseline"      # "baseline" | "MobileNet_V3" | "EfficientNet_V3"

SAVE_DIR  = "conditional_aug"
ROUND_TAG = "round_04"

# Generation speed/quality
GENERATOR_MODEL    = "sd15"         # "sd15" (fast) or "sdxl"
NUM_STEPS          = 20
GUIDANCE_SCALE     = 7.5
TORCH_DTYPE        = torch.float16
DEVICE             = "cuda" if torch.cuda.is_available() else "cpu"
GLOBAL_SEED        = 12345
BATCH_N            = 2

# DeepFace thresholds & detector
GENDER_CONF_THRESHOLD  = 0.90
EMOTION_CONF_THRESHOLD = 0.90
DETECTOR_BACKEND       = "mtcnn"
ENFORCE_TARGET_MATCH   = True

# Labels
EMOTIONS = ["angry","disgust","fear","happy","sad","surprise","neutral"]

# Training CSV schema
SCHEMA_COLS = [
    "filename","dataset","split",
    "gender_pred","gender","dominant_emotion","gender_emotion_class",
    "angry","disgust","fear","happy","sad","surprise","neutral",
    "face_x","face_y","face_w","face_h","face_area","face_aspect_ratio",
    "brightness","sharpness","valence","arousal"
]

# Circumplex (for metadata)
CIRCUMPLEX_VA = {
    "angry":   (-0.7, 0.8), "disgust": (-0.6, 0.5), "fear": (-0.7, 0.9),
    "happy":   ( 0.8, 0.6), "sad":     (-0.8, 0.3), "surprise": (0.4, 0.9),
    "neutral": ( 0.0, 0.0),
}

#
# ONE-SHOT AUGMENTATION PLAN (for each dataset these counts would change based on the heuristics in the cell above : counts)
#
AUG_PLAN = {
    "baseline": [
        ("fear",     "Man",   50),
        ("sad",      "Woman", 50),
        ("angry",    "Woman", 100),
        ("happy",    "Woman", 300),
        ("surprise", "Woman", 50),
        ("neutral",  "Woman", 0),
        ("disgust",  "Woman", 0),
    ],
    "MobileNet_V3": [
        ("fear",     "Man",   50),
        ("happy",    "Man",   50),
        ("sad",      "Woman", 50),
        ("angry",    "Woman", 100),
        ("surprise", "Woman", 50),
        ("neutral",  "Woman", 0),
        ("disgust",  "Woman", 0),
    ],
    "EfficientNet_B2": [
        ("happy",    "Man",   100),
        ("fear",     "Man",   50),
        ("sad",      "Woman", 50),
        ("angry",    "Woman", 100),
        ("surprise", "Woman", 50),
        ("neutral",  "Woman", 0),
        ("disgust",  "Woman", 0),
    ],
}

# helpers
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def pillow_to_bgr(pil_image):
    rgb = np.array(pil_image.convert("RGB"))
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

def build_prompt(gender: str, emotion: str) -> Tuple[str, str]:
    pos = (f"portrait photo of a {gender.lower()} with a clear {emotion.lower()} facial expression, "
           f"studio lighting, professional photography, sharp focus, detailed skin texture")
    neg = "blurry, distorted, deformed, cartoon, low-resolution, extra limbs, bad anatomy, jpeg artifacts"
    return pos, neg

def brightness_hsv_v(bgr: np.ndarray) -> float:
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    return float(hsv[..., 2].mean())

def sharpness_laplacian(bgr: np.ndarray) -> float:
    return float(cv2.Laplacian(bgr, cv2.CV_64F).var())

def compute_valence_arousal(emo_probs: Dict[str,float]) -> Tuple[float, float]:
    v_sum = a_sum = w_sum = 0.0
    for k, p in emo_probs.items():
        k = k.lower()
        if k in CIRCUMPLEX_VA:
            v, a = CIRCUMPLEX_VA[k]
            v_sum += v * float(p)
            a_sum += a * float(p)
            w_sum += float(p)
    return (v_sum / w_sum, a_sum / w_sum) if w_sum > 0 else (0.0, 0.0)

def _unique_name(dst_dir: str, base: str) -> str:
    stem, ext = os.path.splitext(base)
    cand = base; k = 1
    while os.path.exists(os.path.join(dst_dir, cand)):
        cand = f"{stem}_r{k:04d}{ext}"
        k += 1
    return cand

def analyze_and_crop_bgr(bgr_img: np.ndarray):
    try:
        res_all = DeepFace.analyze(
            img_path=bgr_img,
            actions=["emotion","gender"],
            detector_backend=DETECTOR_BACKEND,
            enforce_detection=True
        )
        res = res_all[0] if isinstance(res_all, list) else res_all

        dom_emotion = str(res["dominant_emotion"]).lower()
        emo_probs   = {str(k).lower(): float(v) for k, v in res["emotion"].items()}
        emo_conf    = float(emo_probs.get(dom_emotion, 0.0))

        dom_gender  = str(res["dominant_gender"])
        gen_probs   = {str(k): float(v) for k, v in res["gender"].items()}
        gen_conf    = float(gen_probs.get(dom_gender, 0.0))

        region = res.get("region", {})
        x, y, w, h = int(region.get("x", 0)), int(region.get("y", 0)), int(region.get("w", 0)), int(region.get("h", 0))
        if w <= 0 or h <= 0:
            return False, None, {"reason": "empty_crop"}

        H, W = bgr_img.shape[:2]
        x, y = max(0, x), max(0, y)
        x2, y2 = min(W, x + w), min(H, y + h)
        face = bgr_img[y:y2, x:x2]
        if face.size == 0:
            return False, None, {"reason": "empty_crop"}

        meta = {
            "dominant_emotion": dom_emotion,
            "emotion_probs": emo_probs,
            "emotion_conf": emo_conf,
            "dominant_gender": dom_gender,
            "gender_probs": gen_probs,
            "gender_conf": gen_conf,
            "region": (x, y, w, h)
        }
        return True, face, meta
    except Exception as e:
        return False, None, {"reason": f"deepface_error: {type(e).__name__}: {e}"}

def passes_thresholds(meta: Dict) -> bool:
    return (meta.get("gender_conf", 0.0)  >= GENDER_CONF_THRESHOLD) and \
           (meta.get("emotion_conf", 0.0) >= EMOTION_CONF_THRESHOLD)

def matches_target(meta: Dict, target_gender: str, target_emotion: str) -> bool:
    if not ENFORCE_TARGET_MATCH:
        return True
    return (str(meta.get("dominant_gender","")).lower() == target_gender.lower()) and \
           (str(meta.get("dominant_emotion","")).lower() == target_emotion.lower())

#
# LOAD GENERATOR
#
print("Loading generator...")
if GENERATOR_MODEL.lower() == "sd15":
    MODEL_ID = "runwayml/stable-diffusion-v1-5"
    pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=TORCH_DTYPE).to(DEVICE)
else:
    MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
    pipe = StableDiffusionXLPipeline.from_pretrained(MODEL_ID, torch_dtype=TORCH_DTYPE, variant="fp16").to(DEVICE)

pipe.enable_attention_slicing()
try:
    pipe.enable_xformers_memory_efficient_attention()
except Exception:
    pass

#
# RESUME STATE
#
round_root = os.path.join(SAVE_DIR, ROUND_TAG, SELECT_DATASET, SELECT_MODEL)
ensure_dir(round_root)

resume_csv = os.path.join(round_root, "resume_manifest.csv")
done = set(pd.read_csv(resume_csv)["src_key"].astype(str).tolist()) if os.path.exists(resume_csv) else set()
print(f"Resume: {len(done)} items already saved")

def append_resume(src_key: str):
    row = pd.DataFrame([{"src_key": src_key}])
    row.to_csv(resume_csv, mode="a", header=not os.path.exists(resume_csv), index=False)

#
# PLAN → FLAG LIST
#
if SELECT_MODEL not in AUG_PLAN:
    raise ValueError(f"SELECT_MODEL '{SELECT_MODEL}' not in AUG_PLAN keys {list(AUG_PLAN.keys())}")

flag_rows: List[Tuple[str, str, int]] = [(emo, g, int(n)) for (emo, g, n) in AUG_PLAN[SELECT_MODEL] if int(n) > 0]
if not flag_rows:
    print("No augmentation requested by plan. Exiting.")

print("Augmentation plan:", flag_rows)

#
# GENERATE → VERIFY/CROP(224×224) → SAVE
#
manifest_rows = []

def save_row_and_image(face_bgr, out_dir, fname, meta, emotion, gender):
    ensure_dir(out_dir)

    #  FORCE 224×224 FOR CKplus
    face_bgr = cv2.resize(face_bgr, (224, 224), interpolation=cv2.INTER_AREA)

    fpath = os.path.join(out_dir, fname)
    cv2.imwrite(fpath, face_bgr)

    fx, fy, fw, fh = meta["region"]
    face_area   = float(fw * fh)
    face_aspect = round(fw / fh, 4) if fh > 0 else np.nan

    bright = round(brightness_hsv_v(face_bgr), 4)
    sharp  = round(sharpness_laplacian(face_bgr), 4)

    probs = {k: float(meta["emotion_probs"].get(k, 0.0)) for k in EMOTIONS}
    v, a = compute_valence_arousal(probs)

    manifest_rows.append({
        "filename": fname,
        "dataset": SELECT_DATASET,
        "split": "train",
        "gender_pred": meta["dominant_gender"],
        "gender": gender,
        "dominant_emotion": emotion,
        "gender_emotion_class": f"{emotion}_{gender}",
        "angry": probs["angry"], "disgust": probs["disgust"], "fear": probs["fear"],
        "happy": probs["happy"], "sad": probs["sad"], "surprise": probs["surprise"], "neutral": probs["neutral"],
        "face_x": int(fx), "face_y": int(fy), "face_w": int(fw), "face_h": int(fh),
        "face_area": face_area, "face_aspect_ratio": face_aspect,
        "brightness": bright, "sharpness": sharp,
        "valence": round(v,4), "arousal": round(a,4),
    })

for (emotion, under_gender, n_target) in flag_rows:
    class_dir = os.path.join(round_root, emotion, under_gender)
    ensure_dir(class_dir)

    pos, neg = build_prompt(under_gender, emotion)

    saved = 0
    tries = 0
    HARD_TRY_CAP = n_target * 6

    pbar = tqdm(total=n_target, desc=f"{under_gender}-{emotion}")
    while saved < n_target and tries < HARD_TRY_CAP:
        this_batch = min(BATCH_N, n_target - saved)
        gen = torch.Generator(device=DEVICE).manual_seed(GLOBAL_SEED + tries)

        if GENERATOR_MODEL.lower() == "sd15":
            out = pipe(prompt=[pos]*this_batch, negative_prompt=[neg]*this_batch,
                       guidance_scale=GUIDANCE_SCALE, num_inference_steps=NUM_STEPS,
                       generator=gen)
            pil_list = out.images
        else:
            out = pipe(prompt=[pos]*this_batch, negative_prompt=[neg]*this_batch,
                       guidance_scale=GUIDANCE_SCALE, generator=gen)
            pil_list = out.images

        for pil_img in pil_list:
            tries += 1
            src_key = f"{emotion}_{under_gender}_{tries}"
            if src_key in done:
                continue

            bgr = pillow_to_bgr(pil_img)
            ok, face_bgr, meta = analyze_and_crop_bgr(bgr)
            if not ok:                      append_resume(src_key); continue
            if not passes_thresholds(meta): append_resume(src_key); continue
            if not matches_target(meta, under_gender, emotion): append_resume(src_key); continue

            fname = f"{emotion}_{under_gender}_{saved}.png"
            save_row_and_image(face_bgr, class_dir, fname, meta, emotion, under_gender)
            append_resume(src_key)

            saved += 1
            pbar.update(1)
            if saved >= n_target:
                break

    pbar.close()
    if saved < n_target:
        print(f" {under_gender}-{emotion}: verified {saved}/{n_target} within try cap ({HARD_TRY_CAP}).")

# Save a manifest (pre-merge)
if manifest_rows:
    pre_manifest = os.path.join(round_root, f"manifest_{SELECT_DATASET}_{SELECT_MODEL}_verified.csv")
    pd.DataFrame(manifest_rows).to_csv(pre_manifest, index=False)
    print(f"Pre-merge manifest written: {pre_manifest}")
else:
    print("No verified crops produced. Exiting early.")

#
# MERGE INTO TRAIN
#
def _unique_name(dst_dir: str, base: str) -> str:
    stem, ext = os.path.splitext(base)
    cand = base; k = 1
    while os.path.exists(os.path.join(dst_dir, cand)):
        cand = f"{stem}_r{k:04d}{ext}"
        k += 1
    return cand

def merge_manifest_into_train(
    manifest_df: pd.DataFrame,
    train_csv_path: str,
    train_img_dir: str,
    gen_root: str,
    dataset: str
):
    if manifest_df is None or manifest_df.empty:
        print("No rows to merge into train."); return

    if os.path.exists(train_csv_path):
        train_df = pd.read_csv(train_csv_path)
    else:
        train_df = pd.DataFrame(columns=SCHEMA_COLS)

    for col in SCHEMA_COLS:
        if col not in train_df.columns:
            default = "train" if col == "split" else (dataset if col == "dataset" else np.nan)
            train_df[col] = default

    appended_rows, copied = [], 0

    for _, r in manifest_df.iterrows():
        src = os.path.join(gen_root, r["dominant_emotion"], r["gender"], r["filename"])
        if not os.path.exists(src):
            continue

        new_name = _unique_name(train_img_dir, r["filename"])
        dst = os.path.join(train_img_dir, new_name)
        shutil.copy2(src, dst)
        copied += 1

        row = r.copy()
        row["filename"] = new_name
        row["split"]    = "train"
        row["dataset"]  = dataset
        appended_rows.append(row)

    if appended_rows:
        add_df = pd.DataFrame(appended_rows).reindex(columns=SCHEMA_COLS)
        merged = pd.concat([train_df, add_df], ignore_index=True)[SCHEMA_COLS]
        merged.to_csv(train_csv_path, index=False)
        print(f"Copied {copied} images → {train_img_dir}")
        print(f"Appended {len(add_df)} rows to CSV → {train_csv_path}")
    else:
        print("No files copied; nothing appended to train CSV.")

pre_manifest_path = os.path.join(round_root, f"manifest_{SELECT_DATASET}_{SELECT_MODEL}_verified.csv")
if os.path.exists(pre_manifest_path):
    manifest_df = pd.read_csv(pre_manifest_path)
    merge_manifest_into_train(
        manifest_df=manifest_df,
        train_csv_path=TRAIN_CSV,
        train_img_dir=TRAIN_IMG_DIR,
        gen_root=round_root,
        dataset=SELECT_DATASET
    )

# Optional: sanity_check_csv_vs_files(TRAIN_CSV, TRAIN_IMG_DIR)